<a href="https://colab.research.google.com/github/ubaldinho/Hello_World/blob/main/PW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Work : Secure Federated Learning

Federated Learning (FL) is a machine learning framework that enables $K \in \mathbb{N}^*$ participants to collaboratively train a model $M^G$ across $R$ rounds of exchange while maintaining the privacy of their data $D^k$. In the client–server model of FL, the server initializes the global model $M^G_0$. At each round $t$, the global model $M^G_t$ is distributed to a subset $S_t \subseteq {1, \dots, K}$ consisting of $C \times K$ randomly selected clients, where $C \in (0,1]$. Each client $k \in S_t$ trains the model locally using its private dataset $D^k$ and sends its updated model $M_{t+1}^k$ back to the server. The server then aggregates these updates to construct the new global model $M_{t+1}^G$. This process repeats until $R$ rounds are completed ($t = R$).

In [ ]:
!pip install tenseal

In [ ]:
from src.train import train
from src.data_splitter import data_splitter
from src.metric import accuracy

import torch
from torch import nn
from copy import deepcopy
from torch import optim

import tenseal as ts

## Understanding the Client-Server Algorithm

In this section, we implement the Federated Learning algorithm using FedAvg. We use the CIFAR-10 dataset and the ResNet-18 model. The server model is trained for 44 rounds, after which one round of training is performed on the clients.


In [ ]:
train_loaders, size, test_loader = # TODO (a) : Load the data using the data_splitter function

In [ ]:
server_44 = torch.hub.load('pytorch/vision', 'resnet18', weights=None) # Load the model architecture
server_44.fc = nn.Linear(server_44.fc.in_features, 10)
server_44. # TODO (b) : Load the model weights of the server at round 44
server_44.to("cuda")
print("Server model at round 44 loaded")

In [ ]:
accuracy(server_44, test_loader)

In the next cell, we define five clients by copying the server model.

In [ ]:
clients = []
for i in range(5):
    client = deepcopy(server_44)
    clients.append(client)

Before launching the FL algorithm, we need to define the FedAvg function, which is defined as follows:
$$ W^G_{t+1} = \sum_{k \in S_t} \frac{n_k}{n} W^k_{t+1} $$
where $W^k_{t+1}$ denotes the weights of client $k$ at round $t+1$, $n_k$ is the size of client $k$’s dataset, and $n$ is the total size of the datasets of all clients.

In [ ]:
def fed_avg(server, clients):
    with torch.no_grad():
        server_next = deepcopy(server)
        server_dict = server_next.state_dict()
        for name_server in server_dict.keys():
            server_dict[name_server].zero_()
            for client in clients:
                if client.state_dict()[name_server].dtype is torch.long:
                    weight = (
                        # todo (d) : Compute the weight of the client's model
                    ) * client.state_dict()[name_server].clone().detach()
                    weight = weight.long()

                else:
                    weight = (
                        # todo (d) : Compute the weight of the client's model
                    ) * client.state_dict()[name_server].clone().detach()

                server_dict[name_server].add_(weight)
    return server_next

Now, we can launch the FL algorithm by performing one round of training on the clients.

In [ ]:
def one_round():
    clients = []
    for i in range(5):
        print("Local Training on client", i)
        client = deepcopy(server_44)
        # TODO (d) : Train the client's model using the train function and store it in the clients list
    server_next = # TODO (d) : Aggregate the client's model using the FedAvg algorithm
    # TODO (e) : Evaluate the accuracy of the server model after aggregation

## Secure Aggregation using TenSEAL

In this section, we implement secure aggregation using the TenSEAL library. We use the CKKS scheme to encrypt the last layer of each client’s model and then aggregate the encrypted layers. Finally, we decrypt the aggregated layer to obtain the final result.

In [ ]:
server_44 = torch.hub.load('pytorch/vision', 'resnet18', weights=None) # Load the model architecture
server_44.fc = nn.Linear(server_44.fc.in_features, 10)
server_44. # TODO (b) : Load the model weights of the server at round 44
server_44.to("cuda")
print("Server model at round 44 loaded")

First, we need to define the encryption context using the CKKS scheme.

In [ ]:
ctx = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
ctx.global_scale = pow(2, 40)
ctx.generate_galois_keys()

Then, we define the function that encrypts the last layer of each client’s model.

In [ ]:
def encrypt_last_layer(clients, ctx):
    encrypted_last_layers = []
    for i in range(5):
        encrypted_last_layers.append(# TODO (f) : Encrypt the last layer of the client's model
    return encrypted_last_layers

In [ ]:
encrypted_last_layers = encrypt_last_layer(clients, ctx)

Now, we can aggregate the encrypted last layers and decrypt the aggregated layer to obtain the final result.

In [ ]:
# TODO (g) : Aggregate the encrypted last layers

In [ ]:
# TODO (h) : Decrypt the aggregated last layer and print the result

You can compare the result with the aggregation of the clients’ models without encryption:

In [ ]:
aggregated_last_layer = 0
for i in range(5):
    aggregated_last_layer += (1/5) + clients[i].fc.bias.cpu().detach()
print(aggregated_last_layer)

## Byzantine Attack

In this section, we implement various Byzantine attacks that aim to compromise the federated learning process by sending malicious updates to the server.

In [ ]:
server_44 = torch.hub.load('pytorch/vision', 'resnet18', weights=None) # Load the model architecture
server_44.fc = nn.Linear(server_44.fc.in_features, 10)
server_44. # TODO (b) : Load the model weights of the server at round 44
server_44.to("cuda")
print("Server model at round 44 loaded")

The next cell contains all the Byzantine attacks that we implement.  
The Byzantine attacks are defined as follows:

- **Lazy Attack:** A client sends arbitrary values (e.g., random or malformed updates).  
- **Same Attack:** A client sends identical values for all parameters.  
- **Sign Attack:** A client multiplies all weights by a scalar $\alpha$ (i.e., scales the model).  
- **Noise Attack:** A client adds random noise to the weights (sampled from a chosen distribution, e.g., $\mathcal{N}(0,\sigma^2)$).  

Finally, we define the filter (defense) that will be used to detect and

In [ ]:
def BA_Lazy(server):
    with torch.no_grad():
        server_next = deepcopy(server)
        server_dict = server_next.state_dict()
        for name_server in server_dict.keys():
            server_dict[name_server]. # TODO (i) : Fill the server's model with a what you want
    return server_next

# Byzantine Attack
def BA_Same(server):
    with torch.no_grad():
        server_next = deepcopy(server)
        server_dict = server_next.state_dict()
        for name_server in server_dict.keys():
            server_dict[name_server]. # TODO (j) : Fill the server's model with the same value
    return server_next

def BA_Sign(server):
    with torch.no_grad():
        server_next = deepcopy(server)
        server_dict = server_next.state_dict()
        for name_server in server_dict.keys():
            server_dict[name_server]. # TODO (j) : Multiply all the weights by a value alpha
    return server_next

def BA_Noise(server):
    with torch.no_grad():
        server_next = deepcopy(server)
        server_dict = server_next.state_dict()
        for name_server in server_dict.keys():
            if server_dict[name_server].dtype is not torch.long:
                server_dict[name_server]. # TODO (j) : Add some noise to the weights
    return server_next

def filter(client):
    # TODO (k) : Filter the client's model
    return True

In [ ]:
def one_round_attack(server):
    # Clients Side
    clients = []
    for i in range(5):
        if i == 4:
            print("Malicious Client", i)
            client = # TODO (k) : Apply the Byzantine Attack on the server
        else:
            print("Local Training on client", i)
            client = deepcopy(server)
            train(client, train_loaders[i], test_loader, 1)
        clients.append(client)

    # Server Side
    for i in range(5):
        print("Client", i)
        # TODO (l) : Filter the client's model
    server_next = fedavg(server_44, clients)
    print("Server Accuracy at round 45 ",accuracy(server_next, test_loader))

In [ ]:
one_round_attack(server_44)

## Protect the Model's IP using Watermarking

In this section, we implement the model watermarking technique defined by Uchida et al. To simplify the implementation, we assume that the model is watermarked by a single client.

In [ ]:
# Model Watermarking
server_44 = torch.hub.load('pytorch/vision', 'resnet18', weights=None)  # Load the model architecture
server_44.fc = nn.Linear(server_44.fc.in_features, 10)
server_44. # TODO (b) : Load the model weights of the server at round 44
server_44.to("cuda")
print("Server model at round 44 loaded")

Before watermarking the model, we examine the limitation of a common method used to evaluate whether two models are identical in open-source platforms. In the next cells, we compute the hash of the last layer of the server model.

In [ ]:
server_44.fc.bias

In [ ]:
accuracy(server_44, test_loader)

In [ ]:
tensor_to_str = ''.join(str(x.item())+" " for x in server_44.fc.bias)
print(tensor_to_str)
print("Hash of the layer :", hash(tensor_to_str))

Let’s add a small perturbation to the last layer of the server model and then compute the hash again.

In [ ]:
with torch.no_grad():
    server_44.fc.bias.add_(1e-3)

In [ ]:
accuracy(server_44, test_loader)

In [ ]:
tensor_to_str = ''.join(str(x.item())+" " for x in server_44.fc.bias)
print(tensor_to_str)
print("Hash of the layer :", hash(tensor_to_str))

As you can see, the hash is different while the accuracy remains the same. This means that an attacker can bypass this simple method of verifying whether two models are identical.

Now let’s implement the watermarking technique proposed by Uchida *et al.*  
This technique consists of embedding a secret message in a layer using the following methodology:

1. **Secret generation:** Generate a secret key $K$ and a message $b$.  
2. **Parameter selection:** Select the parameters `"fc.weight"` and compute the mean along the columns to obtain a vector $w$.  
3. **Projection:** Project the vector $w$, in which we want to embed $b$, using the secret key $K$ as follows:  
   $$
   y = K w
   $$  
4. **Extraction:** Apply the Sigmoid function to obtain the extracted message $b'$:  
   $$
   b' = \sigma(y)
   $$  
5. **Loss computation:** Compute the binary cross-entropy loss between the extracted message $b'$ and the original message $b$:  
   $$
   L = \text{BCELoss}(b', b)
   $$

In [ ]:
secret_key = torch.randn((256,512), device="cuda")
message = torch.randint(2, (256,), device="cuda").float()
def train(model, train_set, test_set, epoch_max):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    criterion_watermark = # TODO (o) : Define the criterion for the watermark
    alpha = 5e-1

    for epoch in range(epoch_max):
        accumulate_loss = 0

        for inputs, outputs in train_set:
            optimizer.zero_grad(set_to_none=True)

            inputs = inputs.to("cuda")

            outputs = outputs.to("cuda")

            outputs_predicted = model(inputs)

            loss_main = criterion(outputs_predicted, outputs)

            extracted_message = # TODO (n) : Extract the message from the model

            loss_watermark  = # TODO (o) : Compute the loss of the watermark

            loss = loss_main + (alpha * loss_watermark)

            loss.backward()

            optimizer.step()

            accumulate_loss += loss.item()
        print(f"Epoch : {epoch}")
        print(accuracy(model, test_set))
        print("Bit Error Rate : ", # TODO (p) : Compute the Bit Error Rate